# Generate split training and test dataset for k-fold cross validation with Differential Privacy

Due to limited memory in a machine, we must divide the dataset into training and test datasets and save them with the .h5ad extension. Since we apply k-fold cross-validation, there will be k datasets for training and testing; thus, the total number of split datasets is 10 (5 for training and 5 for testing). <br><br>

Apply Differential Privacy as adding laplace noise to the `adata.X`. This achieves (ε, 0)-DP.

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import argparse
import warnings
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split
from numpy.random import default_rng
from scipy.stats import laplace


np.random.seed(41)
sc.settings.verbosity = 0
RANDOM_SEED = 110011

In [2]:
def n_random_sampling(adata, n=1, max_samples_covid=128, max_samples_control=28, RANDOM_SEED=110011):
    # Set the maximum number of samples for each severity group
    max_samples_covid = max_samples_covid
    max_samples_control = max_samples_control
    
    # Get the samples for each severity group
    severe_critical_samples = adata.obs[adata.obs['CoVID-19 severity'] == 'severe/critical']['sampleID_label'].unique()
    mild_moderate_samples = adata.obs[adata.obs['CoVID-19 severity'] == 'mild/moderate']['sampleID_label'].unique()
    control_samples = adata.obs[adata.obs['CoVID-19 severity'] == 'control']['sampleID_label'].unique()

    # Get lists of all the samples for each fold
    # key: fold number, value: list of samples
    k_sets_samples_severe_critical = dict() 
    k_set_samples_mild_moderate = dict()
    k_sets_samples_control = dict()
    
    for i in range(n):
        rng = default_rng(seed=RANDOM_SEED)
        # Randomly select max_samples samples for each severity group
        clip_samples_severe_critical = rng.choice(a=severe_critical_samples, size=max_samples_covid, replace=False, shuffle=True)
        clip_samples_mild_moderate = rng.choice(a=mild_moderate_samples, size=max_samples_covid, replace=False, shuffle=True)
        clip_samples_control = rng.choice(a=control_samples, size=max_samples_control, replace=False, shuffle=True) # There are only 28 control samples.
        
        k_sets_samples_severe_critical[i] = clip_samples_severe_critical
        k_set_samples_mild_moderate[i] = clip_samples_mild_moderate
        k_sets_samples_control[i] = clip_samples_control
    
    # Make assert if each value of k_sets_samples_severe_critical are the same.
    for i in range(n):
        for j in range(i+1, n):
            if set(k_sets_samples_severe_critical[i]) == set(k_sets_samples_severe_critical[j]):
                raise AssertionError("Not all values in k_sets_samples_severe_critical are the same")
                
    # Make assert if each value of k_set_samples_mild_moderate are the same.
    for i in range(n):
        for j in range(i+1, n):
            if set(k_set_samples_mild_moderate[i]) == set(k_set_samples_mild_moderate[j]):
                raise AssertionError("Not all values in k_set_samples_mild_moderate are the same")

    return k_sets_samples_severe_critical, k_set_samples_mild_moderate, k_sets_samples_control

In [3]:
def apply_DP(X, epsilon):
    '''
    Apply differential privacy to the data
    X: numpy array
    sensitivity: float
    epsilon: float
    return: numpy array
    '''
    print("Start applying DP...")
    # Sensitivity calculation (cell-level)
    sensitivity = np.max(np.sum(X, axis=1))
    print("Sensitivity: ", sensitivity)
    print("Epsilon: ", epsilon)
    # Laplace noise scale parameter (cell-level)
    scale_cells = sensitivity / epsilon

    # Generate Laplace noise
    laplace_noise = laplace.rvs(scale=scale_cells, size=X.shape)
    # Apply input perturbation
    perturbed_matrix = X + laplace_noise
    print("Done!")
    
    return perturbed_matrix
    

In [4]:
# Read adata
num_dataset_sampling=1
num_kfold=1
print("Reading a h5ad data file...")
adata = sc.read_h5ad('./data/GSE_158055_COVID19_ALL.h5ad')

# Delete objects to prepare for splitting the adata to train and test.
del adata.uns['neighbors']
del adata.uns['pca']
del adata.obsm['X_pca']
del adata.obsm['X_tsne']
del adata.obsp

# Change the type of the sampleID and CoVID-19 severity to string.
adata.obs['sampleID'] = adata.obs['sampleID'].astype('str')
adata.obs['CoVID-19 severity'] = adata.obs['CoVID-19 severity'].astype('str')

# Make a new column that has the combined information of the sampleID and CoVID-19 severity.
adata.obs['sampleID_label'] = adata.obs['sampleID'] + '_' + adata.obs['CoVID-19 severity']

# Dataset class distribution: “severe/critical” : 134, “mild/moderate”: 122, and “control”: 28.
print("Clipping the adata due to the memory limitation...")
max_samples_covid = 60
max_samples_control = 28 # There is only 28 control samples.
k_sets_samples_severe_critical, k_set_samples_mild_moderate, k_sets_samples_control = \
                                                                n_random_sampling(adata, num_dataset_sampling, max_samples_covid, max_samples_control, RANDOM_SEED)
                                                                
print("Sets of samples for severe/critical: ", k_sets_samples_severe_critical)
print("Sets of samples for mild/moderate: ", k_set_samples_mild_moderate)
print("Sets of samples for control: ", k_sets_samples_control)


Reading a h5ad data file...


### Create adata_train and adata_test for k=0 in k-fold cross validation

In [ ]:
RANDOM_SEED = 110011
k=0

for rep in ['X_pca', 'X_umap']:
    print("----------------Using an input representation: ", rep, " ------------------")
    if rep == 'X_umap':
        rep_name = 'UMAP'
    else:
        rep_name = 'PCA'

    # Apply DP
    adata.X = apply_DP(adata.X, epsilon=1.0)
    
    # Create adata for train and test (k=0).
    start_time = time.time()
    print('Started at: ', start_time)
    print("k_fold cross validation sets (split randomly for train and test)", k, " on the ", i, "st clipped dataset","--------")
    list_samples = list(adata.obs['sampleID_label'].unique())
    y_train, y_test = train_test_split(list_samples, test_size=0.20, random_state=RANDOM_SEED)
    
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)
    adata_train = adata[adata.obs['contain_y_train'] == True,:].copy()
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_test'] = adata.obs['sampleID_label'].isin(y_test)
    adata_test = adata[adata.obs['contain_y_test'] == True,:].copy()
    
    # Delete adata to free up memory.
    #del adata
    
    print("Run PCA and UMAP on the adata_train.")
    # Run PCA and neighbour graph on the adata_train. 
    sc.pp.neighbors(adata_train, n_pcs = 30, n_neighbors = 20) 
    sc.tl.pca(adata_train)
    if rep == 'X_umap':
        print("Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).")
        sc.tl.umap(adata_train)
        sc.tl.ingest(adata_test, adata_train, embedding_method='umap')
    else:
        print("Run sc.tl.ingest on the adata_test by the fit reducer (PCA).")
        sc.tl.ingest(adata_test, adata_train, embedding_method='pca')
    
    # Concatenate adata_train and adata_test into adata
    adata_concat = adata_train.concatenate(adata_test, batch_categories=['ref', 'new'])
    # Change the type of contain_y_train from bool to string.
    adata_concat.obs['contain_y_train'] = adata_concat.obs['contain_y_train'].astype('str')
    adata_concat.obs['contain_y_test'] = adata_concat.obs['contain_y_test'].astype('str')
    # Save the adata_concat to a h5ad file.
    print(f"Saving {k}th adata in {rep_name} rep to a h5ad file...")
    adata_concat.write(f'../../data/k{k}_{rep_name}_adata_GSE_158055_COVID19_DP.h5ad')
    
    print('Ended at: ', time.time())

----------------Using an input representation:  X_pca  ------------------
Start applying DP...
Sensitivity:  8696.728
Epsilon:  0.1


: 

: 

In [4]:
del adata_concat
del adata_train
del adata_test

# Drop the column of contain_y_train and contain_y_test from adata.obs
adata.obs.drop(columns=['contain_y_train', 'contain_y_test'], inplace=True)

### Create adata_train and adata_test for k=1 in k-fold cross validation

In [5]:
RANDOM_SEED = 1234
k=1

for rep in ['X_pca', 'X_umap']:
    print("----------------Using an input representation: ", rep, " ------------------")
    if rep == 'X_umap':
        rep_name = 'UMAP'
    else:
        rep_name = 'PCA'

    # Create adata for train and test (k=1)
    start_time = time.time()
    print("k_fold cross validation sets (split randomly for train and test)", k, " on the ", i, "st clipped dataset","--------")
    list_samples = list(adata.obs['sampleID_label'].unique())
    y_train, y_test = train_test_split(list_samples, test_size=0.20, random_state=RANDOM_SEED)
    
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)
    adata_train = adata[adata.obs['contain_y_train'] == True,:].copy()
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_test'] = adata.obs['sampleID_label'].isin(y_test)
    adata_test = adata[adata.obs['contain_y_test'] == True,:].copy()
    
    # Delete adata to free up memory.
    #del adata
    
    print("Run PCA and UMAP on the adata_train.")
    # Run PCA and neighbour graph on the adata_train. 
    sc.pp.neighbors(adata_train, n_pcs = 30, n_neighbors = 20) 
    sc.tl.pca(adata_train)
    if rep == 'X_umap':
        print("Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).")
        sc.tl.umap(adata_train)
        sc.tl.ingest(adata_test, adata_train, embedding_method='umap')
    else:
        print("Run sc.tl.ingest on the adata_test by the fit reducer (PCA).")
        sc.tl.ingest(adata_test, adata_train, embedding_method='pca')
    
    # Concatenate adata_train and adata_test into adata
    adata_concat = adata_train.concatenate(adata_test, batch_categories=['ref', 'new'])
    # Change the type of contain_y_train from bool to string.
    adata_concat.obs['contain_y_train'] = adata_concat.obs['contain_y_train'].astype('str')
    adata_concat.obs['contain_y_test'] = adata_concat.obs['contain_y_test'].astype('str')
    # Save the adata_concat to a h5ad file.
    print(f"Saving {k}th adata in {rep_name} rep to a h5ad file...")
    adata_concat.write(f'../../data/k{k}_{rep_name}_adata_GSE_158055_COVID19.h5ad')

----------------Using an input representation:  X_pca  ------------------
k_fold cross validation sets (split randomly for train and test) 1  on the  0 st clipped dataset --------
Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (PCA).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 1th adata in PCA rep to a h5ad file...
----------------Using an input representation:  X_umap  ------------------
k_fold cross validation sets (split randomly for train and test) 1  on the  0 st clipped dataset --------
Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 1th adata in UMAP rep to a h5ad file...


In [7]:
del adata_concat
del adata_train
del adata_test

# Drop the column of contain_y_train and contain_y_test from adata.obs
adata.obs.drop(columns=['contain_y_train', 'contain_y_test'], inplace=True)

### Create adata_train and adata_test for k=2 in k-fold cross validation

In [3]:
RANDOM_SEED = 1235
k=2

for rep in ['X_pca', 'X_umap']:
    print("----------------Using an input representation: ", rep, " ------------------")
    if rep == 'X_umap':
        rep_name = 'UMAP'
    else:
        rep_name = 'PCA'

    # Create adata for train and test (k=2)
    start_time = time.time()
    print("k_fold cross validation sets (split randomly for train and test)", k, " on the ", i, "st clipped dataset","--------")
    list_samples = list(adata.obs['sampleID_label'].unique())
    y_train, y_test = train_test_split(list_samples, test_size=0.20, random_state=RANDOM_SEED)
    
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)
    adata_train = adata[adata.obs['contain_y_train'] == True,:].copy()
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_test'] = adata.obs['sampleID_label'].isin(y_test)
    adata_test = adata[adata.obs['contain_y_test'] == True,:].copy()
    
    # Delete adata to free up memory.
    #del adata
    
    print("Run PCA and UMAP on the adata_train.")
    # Run PCA and neighbour graph on the adata_train. 
    sc.pp.neighbors(adata_train, n_pcs = 30, n_neighbors = 20) 
    sc.tl.pca(adata_train)
    if rep == 'X_umap':
        print("Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).")
        sc.tl.umap(adata_train)
        sc.tl.ingest(adata_test, adata_train, embedding_method='umap')
    else:
        print("Run sc.tl.ingest on the adata_test by the fit reducer (PCA).")
        sc.tl.ingest(adata_test, adata_train, embedding_method='pca')
    
    # Concatenate adata_train and adata_test into adata
    adata_concat = adata_train.concatenate(adata_test, batch_categories=['ref', 'new'])
    # Change the type of contain_y_train from bool to string.
    adata_concat.obs['contain_y_train'] = adata_concat.obs['contain_y_train'].astype('str')
    adata_concat.obs['contain_y_test'] = adata_concat.obs['contain_y_test'].astype('str')
    # Save the adata_concat to a h5ad file.
    print(f"Saving {k}th adata in {rep_name} rep to a h5ad file...")
    adata_concat.write(f'../../data/k{k}_{rep_name}_adata_GSE_158055_COVID19.h5ad')

----------------Using an input representation:  X_pca  ------------------
k_fold cross validation sets (split randomly for train and test) 2  on the  0 st clipped dataset --------


/tmp/ipykernel_4150989/197565163.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)


Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (PCA).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 2th adata in PCA rep to a h5ad file...
----------------Using an input representation:  X_umap  ------------------
k_fold cross validation sets (split randomly for train and test) 2  on the  0 st clipped dataset --------
Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 2th adata in UMAP rep to a h5ad file...


In [4]:
del adata_concat
del adata_train
del adata_test

# Drop the column of contain_y_train and contain_y_test from adata.obs
adata.obs.drop(columns=['contain_y_train', 'contain_y_test'], inplace=True)

### Create adata_train and adata_test for k=3 in k-fold cross validation

In [3]:
RANDOM_SEED = 2222
k=3

for rep in ['X_pca', 'X_umap']:
    print("----------------Using an input representation: ", rep, " ------------------")
    if rep == 'X_umap':
        rep_name = 'UMAP'
    else:
        rep_name = 'PCA'

    # Create adata for train and test (k=2)
    start_time = time.time()
    print("k_fold cross validation sets (split randomly for train and test)", k, " on the ", i, "st clipped dataset","--------")
    list_samples = list(adata.obs['sampleID_label'].unique())
    y_train, y_test = train_test_split(list_samples, test_size=0.20, random_state=RANDOM_SEED)
    
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)
    adata_train = adata[adata.obs['contain_y_train'] == True,:].copy()
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_test'] = adata.obs['sampleID_label'].isin(y_test)
    adata_test = adata[adata.obs['contain_y_test'] == True,:].copy()
    
    # Delete adata to free up memory.
    #del adata
    
    print("Run PCA and UMAP on the adata_train.")
    # Run PCA and neighbour graph on the adata_train. 
    sc.pp.neighbors(adata_train, n_pcs = 30, n_neighbors = 20) 
    sc.tl.pca(adata_train)
    if rep == 'X_umap':
        print("Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).")
        sc.tl.umap(adata_train)
        sc.tl.ingest(adata_test, adata_train, embedding_method='umap')
    else:
        print("Run sc.tl.ingest on the adata_test by the fit reducer (PCA).")
        sc.tl.ingest(adata_test, adata_train, embedding_method='pca')
    
    # Concatenate adata_train and adata_test into adata
    adata_concat = adata_train.concatenate(adata_test, batch_categories=['ref', 'new'])
    # Change the type of contain_y_train from bool to string.
    adata_concat.obs['contain_y_train'] = adata_concat.obs['contain_y_train'].astype('str')
    adata_concat.obs['contain_y_test'] = adata_concat.obs['contain_y_test'].astype('str')
    # Save the adata_concat to a h5ad file.
    print(f"Saving {k}th adata in {rep_name} rep to a h5ad file...")
    adata_concat.write(f'../../data/k{k}_{rep_name}_adata_GSE_158055_COVID19.h5ad')

----------------Using an input representation:  X_pca  ------------------
k_fold cross validation sets (split randomly for train and test) 3  on the  0 st clipped dataset --------


/tmp/ipykernel_358219/2650027277.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)


Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (PCA).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 3th adata in PCA rep to a h5ad file...
----------------Using an input representation:  X_umap  ------------------
k_fold cross validation sets (split randomly for train and test) 3  on the  0 st clipped dataset --------
Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 3th adata in UMAP rep to a h5ad file...


In [4]:
del adata_concat
del adata_train
del adata_test

# Drop the column of contain_y_train and contain_y_test from adata.obs
adata.obs.drop(columns=['contain_y_train', 'contain_y_test'], inplace=True)

### Create adata_train and adata_test for k=4 in k-fold cross validation

In [5]:
RANDOM_SEED = 123445
k=4

for rep in ['X_pca', 'X_umap']:
    print("----------------Using an input representation: ", rep, " ------------------")
    if rep == 'X_umap':
        rep_name = 'UMAP'
    else:
        rep_name = 'PCA'

    # Create adata for train and test (k=2)
    start_time = time.time()
    print("k_fold cross validation sets (split randomly for train and test)", k, " on the ", i, "st clipped dataset","--------")
    list_samples = list(adata.obs['sampleID_label'].unique())
    y_train, y_test = train_test_split(list_samples, test_size=0.20, random_state=RANDOM_SEED)
    
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)
    adata_train = adata[adata.obs['contain_y_train'] == True,:].copy()
    # Make adata for train/existing data.
    # Make a column that contains bool values based on whether the sample_name holds one of the samples in the y_train..
    adata.obs['contain_y_test'] = adata.obs['sampleID_label'].isin(y_test)
    adata_test = adata[adata.obs['contain_y_test'] == True,:].copy()
    
    # Delete adata to free up memory.
    #del adata
    
    print("Run PCA and UMAP on the adata_train.")
    # Run PCA and neighbour graph on the adata_train. 
    sc.pp.neighbors(adata_train, n_pcs = 30, n_neighbors = 20) 
    sc.tl.pca(adata_train)
    if rep == 'X_umap':
        print("Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).")
        sc.tl.umap(adata_train)
        sc.tl.ingest(adata_test, adata_train, embedding_method='umap')
    else:
        print("Run sc.tl.ingest on the adata_test by the fit reducer (PCA).")
        sc.tl.ingest(adata_test, adata_train, embedding_method='pca')
    
    # Concatenate adata_train and adata_test into adata
    adata_concat = adata_train.concatenate(adata_test, batch_categories=['ref', 'new'])
    # Change the type of contain_y_train from bool to string.
    adata_concat.obs['contain_y_train'] = adata_concat.obs['contain_y_train'].astype('str')
    adata_concat.obs['contain_y_test'] = adata_concat.obs['contain_y_test'].astype('str')
    # Save the adata_concat to a h5ad file.
    print(f"Saving {k}th adata in {rep_name} rep to a h5ad file...")
    adata_concat.write(f'../../data/k{k}_{rep_name}_adata_GSE_158055_COVID19.h5ad')

----------------Using an input representation:  X_pca  ------------------
k_fold cross validation sets (split randomly for train and test) 4  on the  0 st clipped dataset --------


/tmp/ipykernel_871113/206868120.py:19: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['contain_y_train'] = adata.obs['sampleID_label'].isin(y_train)


Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (PCA).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 4th adata in PCA rep to a h5ad file...
----------------Using an input representation:  X_umap  ------------------
k_fold cross validation sets (split randomly for train and test) 4  on the  0 st clipped dataset --------
Run PCA and UMAP on the adata_train.
Run sc.tl.ingest on the adata_test by the fit reducer (UMAP).


/home/dozonok/anaconda3/envs/workshop_scanpy_2/lib/python3.8/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


Saving 4th adata in UMAP rep to a h5ad file...
